In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint

# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import scale as std_scale, robust_scale

# from scipy.signal import find_peaks, find_peaks_cwt, peak_widths

import numpy as np
# from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import matplotlib.animation as ani

import mlgrad.models as models
import mlgrad.regr as regr
import mlgrad.cls as cls
import mlgrad.plots as plots
import mlgrad.funcs as funcs
import mlgrad.funcs2 as funcs2
import mlgrad.loss as loss
import mlgrad.gd as gd
import mlgrad.smooth as smooth

import rampy
import pybaselines
import random

from irsa import io
from irsa.preprocess import despike, utils


# import xarray as xr

import ipywidgets
from IPython.display import display

%matplotlib widget

In [2]:
attrs = io.collect_attr_values(f"/home/intellimath/repos/spectras")
pprint(attrs)

{'вид_бактерий': ['SA'],
 'время_накопления': ['0.5', '1'],
 'дата': ['20231030', '20240701', '20240704', '20240708', '20240709'],
 'дата_подложки': ['231030', '240301', '240625'],
 'количество_спектров_в_серии': ['15', '30'],
 'комментарий': ['0', '24hr_after_drying'],
 'лазер': ['532'],
 'мощность_лазера': ['4.08'],
 'начальная_концентрация_клеток_в_пробе': ['3', '6'],
 'номер_отбора_из_чашки_петри': ['1'],
 'номер_повтора': ['1', '5'],
 'номер_цикла': ['3'],
 'номер_эксперимента_в_цикле': ['1', '3', '5', '6'],
 'отсечки_по_молекулярной_массе': ['less10', 'less30', 'more30'],
 'резистентность': ['R', 'S'],
 'тип_измерения_спектров': ['SE'],
 'тип_подложки': ['Ag#G_Af'],
 'штамм_бактерий': ['191', '606', '92', '93', '95', '96']}


In [3]:
keys = ['штамм_бактерий', 'резистентность', 'начальная_концентрация_клеток_в_пробе', 
        'отсечки_по_молекулярной_массе', 'номер_эксперимента_в_цикле', 'дата']

selectors = {}
for key in keys:
    vals = attrs[key]
    selectors[key] = ipywidgets.SelectMultiple(options=vals, description=key)
options_button = ipywidgets.Button(description="Select")
output = ipywidgets.Output()

options = {}
def onclick_options_button(b):
    for key,sel in selectors.items():
        if sel.value:
            options[key] = sel.value
    with output:
        print(options)

options_button.on_click(onclick_options_button)

options_widgets = list(selectors.values())+[options_button, output]
display(*options_widgets)

SelectMultiple(description='штамм_бактерий', options=('191', '606', '92', '93', '95', '96'), value=())

SelectMultiple(description='резистентность', options=('R', 'S'), value=())

SelectMultiple(description='начальная_концентрация_клеток_в_пробе', options=('3', '6'), value=())

SelectMultiple(description='отсечки_по_молекулярной_массе', options=('less10', 'less30', 'more30'), value=())

SelectMultiple(description='номер_эксперимента_в_цикле', options=('1', '3', '5', '6'), value=())

SelectMultiple(description='дата', options=('20231030', '20240701', '20240704', '20240708', '20240709'), value…

Button(description='Select', style=ButtonStyle())

Output()

In [4]:
options = {'резистентность': ('R',), 'отсечки_по_молекулярной_массе': ('less30',)}
d = io.load_spectras(f"/home/intellimath/repos/spectras", options)

for i, key in enumerate(d):
    entry1 = d[key]
    entry1.crop(40)
    # entry1.use_range(end_index=15)
    # entry1.smooth(tau=4.0)
    entry1.allign_bottom()
    # entry1.remove_overflow_spectras(y_max=2000.0, y_max_count=30)
    # entry1.remove_outlier_spectras(delta=0.10, tau=3.5)
    # entry1.remove_by_zscore_spectras(tau=3.5, max_count=50)
    print(len(entry1.y))
    

file: 3_3_SA_95_R_3_1_less30_1_0
file: 3_3_SA_96_R_3_1_less30_1_0
file: 3_1_SA_96_R_6_1_less30_1_0
file: 3_2_SA_96_R_6_1_less30_1_!24hours after drying!
60
61
61
34


In [5]:
d_keys = list(d.keys())
sel_i = 0
sel = ipywidgets.Select(options=d_keys,  description="Key:", rows=len(d)+1)
sel.layout.width="50%"

i_slider = ipywidgets.IntSlider(min=0, max=len(d[d_keys[0]].y)-1)
i_slider.layout.width="50%"

f_slider = ipywidgets.FloatSlider(value=3.5, min=1.0, max=10.0)
f_slider.layout.width="50%"

def sel_on_value_change(change):
    i_slider.value = 0
    i_slider.max = len(d[sel.value].y)-1

# def i_on_value_change(change):
#     plot_spectra_series(sel.value, i_slider.value, f_slider.value)

# def f_on_value_change(change):
#     plot_spectra_series(sel.value, i_slider.value, f_slider.value)

sel.on_trait_change(sel_on_value_change, name="value")
# i_slider.on_trait_change(i_on_value_change, name="value")
# f_slider.on_trait_change(f_on_value_change, name="value")

# ipywidgets.VBox([sel, i_slider, out])

In [6]:
def plot_spectra_series(key, i, zval):
    plt.figure(figsize=(12,4))
    entry = d[key]
    x = entry.x[i]
    ys = entry.y[i]
    plt.title(f"{key} [{len(entry.y)}] [{len(ys)}]")
    for j, y in enumerate(ys):
        plt.plot(x, y, linewidth=0.5, label=str(j), alpha=0.5)
    ys_m = utils.robust_mean2(ys, tau=zval)
    ys_m_smooth, qvals = smooth.whittaker(ys_m, tau=1.0, h=0.001, n_iter=1000)
    # print(f"qvals: {qvals}")
    plt.plot(x, ys_m, linewidth=2.0, color='r')
    plt.plot(x, ys_m_smooth, linewidth=2.0, color='k')

    ys_max = max(ys_m_smooth) / 2
    bs, _ = pybaselines.whittaker.aspls(ys_m_smooth/ys_max, x_data=x, lam=1.0e5, diff_order=2)
    # def arpls_func(E):
    #     E1 = np.ascontiguousarray(E[E < 0])
    #     if len(E1) == 0:
    #         return np.ones_like(E, "d")
    #     e_m = np.mean(E1)
    #     s_m = np.mean(abs(E1 - e_m))
    #     EE = (E - e_m) / s_m
    #     W = 1/(1 + np.exp(-2*EE))
    #     # print(W)
    #     return W
    # bs, _ = smooth.whittaker(ys_m_smooth/ys_max, tau=1.0e5, weight_func=arpls_func, h=0.001, n_iter=1000)
    bs *= ys_max
    plt.plot(x, bs, linewidth=2.0, color='m')

    dys_m = abs(ys - ys_m_smooth) / (ys_m_smooth + 0.001)
    ds4 = np.nanmedian(abs(ys_m - ys_m_smooth) / (ys_m + 0.001))
    dy_m = np.nanmedian(dys_m, axis=0)
    ds = np.nanmedian(dy_m) 
    ds2 = np.nanpercentile(dy_m, 75)
    
    plt.hlines(0, 0, max(x), colors='k', linestyles='--')
    plt.xlim(-10, max(x)+10)
    plt.ylim(-10, max(ys[0,42:])+100)
        
    plt.minorticks_on()
    plt.tight_layout()
    plt.legend([str(i), f"{ds:.2f} / {ds2:.2f} ... {ds4:.4f}", "baseline"])
    plt.show()
    plt.figure()
    plt.plot(qvals)
    plt.show()

def plot_zscore_series(key, i, zval):
    plt.figure(figsize=(12,2))
    # plt.title(f"Modified z-score: {key}: {i}")
    entry = d[key]
    x = entry.x[i]
    ys = entry.y[i]
    zs = utils.modified_zscore2(ys)
    abs_zs = abs(zs)
    for z in abs_zs:
        plt.plot(x, z, linewidth=0.5, alpha=0.5)
    plt.plot(x, utils.robust_mean2(abs_zs, tau=zval), linewidth=2.0, color='k')
    plt.hlines(0, 0, max(x), colors='k', linestyles='--')
    plt.hlines(zval, 0, max(x), colors='k', linewidth=1.0)
    plt.xlim(-10, max(x)+10)
        
    plt.minorticks_on()
    plt.tight_layout()
    plt.legend(["Modified z-score"])
    plt.xticks([])
    plt.show()

@ipywidgets.interact(key=sel, i=i_slider, f=f_slider, zscore=False)
def plot_spectra_zscore(key, i, f, zscore):
    if i > len(d[key].y)-1:
        i = 0
        i_slider.value=i
        i_slider.max = len(d[key].y)-1

    plot_spectra_series(key, i, f)
    if zscore:
        plot_zscore_series(key, i, f)


interactive(children=(Select(description='Key:', layout=Layout(width='50%'), options=('SA_95_R_less30_3_3_1_20…

In [ ]:
d2 = {}
for i, key in enumerate(d):
    entry2 = d[key]
    d2[key] = entry2.robust_averaging(tau=3.0)


In [ ]:
for i, key in enumerate(d):
    entry3 = d2[key]
    entry3.smooth(methos="irsa", tau=0.01)
    entry3.allign_bottom()
    entry3.normalize_area_under_curve()
    entry3.remove_by_zscore_spectras(tau=3.5, max_count=40)
    entry3.normalize_area_under_curve()
    entry3.subtract_baseline("aspls", pad=10, lam=1.0e5)
    # print(entry3.x.shape, entry3.y.shape)
    # entry3.y = entry3.remove_outlier_spectras(tau=3.5)
    # Is = [i for i,ys in enumerate(entry3.y) if np.any(np.isnan(ys))]
    # print(Is)            

    # # entry3.subtract_baseline("mor", max_half_window=5, min_half_window=1)


In [ ]:
for i, key in enumerate(d):
    entry = d2[key]
    Xs, Ys = entry.x, entry.y

    plt.figure(figsize=(12,4))
    plt.title(key)
    for xs, ys in zip(Xs, Ys):
        plt.plot(xs, ys, linewidth=0.5)
    plt.hlines(0, 0, max(xs), colors='k', linestyles='--')
    plt.xlim(-10, max(xs))
        
    plt.minorticks_on()
    plt.tight_layout()
    # plt.legend()
    plt.show()
    
    # plt.figure(figsize=(12,3))
    # Zs = utils.modified_zscore2(Ys)
    # for xs,zs in zip(Xs,Zs):
    #     plt.plot(xs,abs(zs), linewidth=0.5, marker='s', markersize=1)
    # plt.hlines(3.5, 0, max(xs), colors='k', linestyle='--')
    # plt.minorticks_on()
    # plt.tight_layout()
    # plt.show()
    

In [ ]:
from sklearn.decomposition import PCA

plt.figure(figsize=(12,6))
for i, key in enumerate(d):
    entry = d2[key]
    Xs, Ys = entry.x, entry.y

    pca = PCA(3)
    pca.fit(Ys)
    Us = pca.transform(Ys)

    plt.scatter(Us[:,1], Us[:,2], label=key)

plt.minorticks_on()
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
for i, key in enumerate(d):
    entry = d2[key]
    Xs, Ys = entry.x, entry.y

    # if "_R_" in key:
    #     if "95" in key:
    #         color='r'
    #     elif "96" in key:
    #         color='Brown'
    # elif "_S_" in key:
    #     if "92" in key:
    #         color='m'
    #     elif "93" in key:
    #         color='b'
    #     else:
    #         color='b'

    if "_R_" in key:
        color='r'
    elif "_S_" in key:
        color='b'
    else:
        color='m'
        
    plt.plot(np.mean(Xs, axis=0), np.mean(Ys, axis=0), linewidth=1.0, color=color, label=key)
    # plt.plot(np.mean(Ys, axis=0), linewidth=1.0, label=key)
    
# plt.hlines(0, 0, len(Ys[0]), colors='k')
# plt.xlim(-10,1000)
        
plt.minorticks_on()
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
# import numpy as np
# import mlgrad.inventory as inventory

In [ ]:
# a = np.array([[10, 7, 4], [3, 2, 1]], "d")
# print(a)

In [ ]:
#np.median(a, axis=0)

In [ ]:
#inventory.median_2d_t(a)